In [11]:
import os
import glob

import librosa
import torchaudio
import soundfile as sf
import pandas as pd
# from src.data.preproc import convert_libri
from src.data.util import log_compress, read_audioset_csv
from src.data.vad import trim_speech

In [19]:
import soundfile as sf
import simpleaudio as sa
def play_audio(path):
    data, samplerate = sf.read(path)
    data = (data * 32767).astype('int16')
    play_obj = sa.play_buffer(
        data,
        num_channels=1 if len(data.shape) == 1 else data.shape[1],
        bytes_per_sample=2,
        sample_rate=samplerate
    )
    play_obj.wait_done()

In [ ]:
import os
import tarfile
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

INPUT_DIR = Path('/Volumes/LKieuData/AudioSet/data/')
OUTPUT_DIR = Path('/Volumes/LKieuData/AudioSet/data/extracted/')  # <-- Change this to your desired output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def extract_tar(tar_path):
    tar_path = Path(tar_path)
    folder_name = tar_path.stem  # filename without extension
    dest_dir = OUTPUT_DIR / folder_name
    if dest_dir.exists():
        print(f'Folder {dest_dir} already exists. Skipping...')
        return
    dest_dir.mkdir(parents=True, exist_ok=True)

    with tarfile.open(tar_path) as tar:
        tar.extractall(path=dest_dir)
    print(f'Extracted {tar_path.name} to {dest_dir}')

tar_files = list(INPUT_DIR.glob('*.tar'))

# with ThreadPoolExecutor(max_workers=4) as executor:
#     executor.map(extract_tar, tar_files)

for file in tar_files:
    extract_tar(file)
    #delete tar file
    os.remove(file)

Folder /Volumes/LKieuData/AudioSet/data/extracted/bal_train06 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train040 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train057 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/bal_train05 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train018 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train039 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train000 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train046 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/eval02 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train004 already exists. Skipping...
Folder /Volumes/LKieuData/AudioSet/data/extracted/unbal_train012 already exists. Ski

In [53]:
# Sample an audio from paths to listen
import random
root_path = '/Users/lkieu/Desktop/Audioset/processed/'
ref = '/Users/lkieu/Desktop/Audioset/audio/bal_train'

paths = glob.glob(os.path.join(root_path, '**', '*.flac'), recursive=True)
sampled_file = random.choice(paths)
ref = os.path.join(ref, sampled_file.split('/')[-1])
print("Sampled:", sampled_file)



Sampled: /Users/lkieu/Desktop/Audioset/processed/7bvl9662aVc.flac


In [1]:
play_audio(sampled_file)
play_audio(ref)

NameError: name 'play_audio' is not defined

In [6]:
from src.data.util import resample

test_flac = 'testVADandLID.m4a'

waveform, sr = torchaudio.load(test_flac, normalize=True)
resample(waveform, sr)


tensor([[0., 0., 0.,  ..., 0., 0., 0.]])

In [4]:
waveform = waveform.squeeze()

In [7]:
from pyannote.audio.pipelines import VoiceActivityDetection
# instantiate the model
from pyannote.audio import Model
model = Model.from_pretrained(
  "pyannote/segmentation-3.0")
pipeline = VoiceActivityDetection(segmentation=model)
HYPER_PARAMETERS = {
  # remove speech regions shorter than that many seconds.
  "min_duration_on": 0.0,
  # fill non-speech regions shorter than that many seconds.
  "min_duration_off": 0.0
}
pipeline.instantiate(HYPER_PARAMETERS)
vad = pipeline({'waveform': waveform, 'sample_rate': sr})
# `vad` is a pyannote.core.Annotation instance containing speech regions
str(vad)

'[ 00:00:00.030 -->  00:00:02.325] 0 SPEECH\n[ 00:00:02.882 -->  00:00:04.739] 0 SPEECH'

In [12]:
vi_segment = '[ 00:00:00.030 -->  00:00:02.325] 0 SPEECH'
en_segment = '[ 00:00:02.882 -->  00:00:04.739] 0 SPEECH'
vi_waveform = trim_speech(waveform, 16000, vi_segment)
en_waveform = trim_speech(waveform, 16000, en_segment)

In [14]:
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor
import torch

model_id = "facebook/mms-lid-256"

processor = AutoFeatureExtractor.from_pretrained(model_id)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_id)
waveform = en_waveform
# English
inputs = processor(waveform.squeeze(), sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

prob = torch.argmax(outputs, dim=-1)
lang_id = prob[0].item()
detected_lang = model.config.id2label[lang_id]
print(detected_lang)
# 'eng'


jav


In [22]:
detected_lang

'lat'

In [33]:
path = '/Users/lkieu/PycharmProjects/PhonemeAwareFoundational/test_audio/balanced_train_segments.csv'
df = read_audioset_csv(path)
df.head(3)

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"[/m/09x0r, /t/dd00088]"
1,--ZhevVpy1s,50.0,60.0,[/m/012xff]
2,--aE2O5G5WE,0.0,10.0,"[/m/03fwl, /m/04rlf, /m/09x0r]"


In [31]:
from src.data.proc_audio_set import convert_audio_set, has_allowed_tag

audio_path = '/Users/lkieu/Desktop/Audioset/audio/bal_train'
paths = convert_audio_set(audio_path)

# Filter for speech tags
df = read_audioset_csv(path)
ytids_to_paths = {
        os.path.splitext(os.path.basename(p))[0]: p for p in paths
    }

ytids = list(ytids_to_paths.keys())
df_filtered = df[df['YTID'].isin(ytids)].copy()
df_filtered['has_allowed_tag'] = df_filtered['positive_labels'].apply(has_allowed_tag)
df_result = df_filtered[df_filtered['has_allowed_tag']==True ].copy()
len(df_result)

534

In [38]:
df_result.head(3)

,YTID,start_seconds,end_seconds,positive_labels,has_allowed_tag
0,--PJHxphWEs,30.0,40.0,"[/m/09x0r, /t/dd00088]",True
2,--aE2O5G5WE,0.0,10.0,"[/m/03fwl, /m/04rlf, /m/09x0r]",True
25,-30H9V1IKps,6.0,16.0,"[/m/07yv9, /m/09x0r, /m/0gvgw0]",True


In [ ]:
# Gather all the flac and json
data_dir = "./LibriLight/small"

def gather_flac_json_pairs(root_dir):
    flac_files = glob.glob(os.path.join(root_dir, '**', '*.flac'), recursive=True)
    pairs = []

    for flac_path in flac_files:
        json_path = os.path.splitext(flac_path)[0] + '.json'
        if os.path.exists(json_path):
            pairs.append((flac_path, json_path))
        else:
            print(f"Warning: No JSON companion for {flac_path}")

    return pairs

pairs = gather_flac_json_pairs(data_dir)

In [ ]:
from collections import defaultdict
import os

def find_duplicate_filenames(paths):
    filename_to_paths = defaultdict(list)

    for path in paths:
        filename = os.path.basename(path)
        filename_to_paths[filename].append(path)

    duplicates = {fname: plist for fname, plist in filename_to_paths.items() if len(plist) > 1}

    for fname, plist in duplicates.items():
        print(f"Duplicate filename: {fname}")
        for p in plist:
            print(f"  {p}")

paths = list(map(lambda x: x[0], pairs))
find_duplicate_filenames(paths)


In [ ]:
# Verify Sampling rate
def get_sample_rate(path):
    metadata = sf.info(path)
    return metadata.samplerate

sample_rate = {}
for path, _ in pairs:
    info = f'{get_sample_rate(path)} hz'
    if info not in sample_rate:
        sample_rate[info] = 1
    else:
        sample_rate[info] += 1
print(sample_rate)

In [ ]:
# Distribution of Voice Activity block length.
# Specific to LibriLight
from collections import Counter
import json
from pathlib import Path
import matplotlib.pyplot as plt

def collect_duration_differences(flac_json_pairs):
    all_durations = []

    for flac_path, json_path in flac_json_pairs:
        with open(json_path, 'r') as f:
            data = json.load(f)

        voice_activity = data.get('voice_activity', [])
        durations = [end - start for start, end in voice_activity]
        all_durations.extend(durations)

    return all_durations

def plot_duration_distribution(durations, bins=50):
    plt.hist(durations, bins=bins, edgecolor='black')
    plt.title('Distribution of Voice Activity Durations')
    plt.xlabel('Duration (seconds)')
    plt.ylabel('Frequency')
    plt.show()

def top_n_durations(durations, n=10, rounding=2):
    rounded_durations = [round(d, rounding) for d in durations]
    counter = Counter(rounded_durations)
    most_common = counter.most_common(n)
    return most_common


durations = collect_duration_differences(pairs)
print(top_n_durations(durations))
print('min: ' + str(min(durations)))
print('max: ' + str(max(durations)))
plot_duration_distribution(durations)

In [ ]:
# What if we allow for silence of 1s

def merge_close_blocks(voice_activity, threshold=1.0):
    if not voice_activity:
        return []

    # Sort by start time just in case
    voice_activity = sorted(voice_activity, key=lambda x: x[0])
    merged = [voice_activity[0]]

    for start, end in voice_activity[1:]:
        last_start, last_end = merged[-1]
        if start - last_end < threshold:
            # Merge intervals
            merged[-1][1] = max(last_end, end)
        else:
            merged.append([start, end])

    return merged

def get_duration_diff_merged(pairs):
    all_durations = []

    for flac_path, json_path in pairs:
        with open(json_path, 'r') as f:
            data = json.load(f)

        voice_activity = data.get('voice_activity', [])
        voice_activity = merge_close_blocks(voice_activity)
        durations = [end - start for start, end in voice_activity]
        all_durations.extend(durations)

    return all_durations

durations = get_duration_diff_merged(pairs)
print(top_n_durations(durations))
print('min: ' + str(min(durations)))
print('max: ' + str(max(durations)))
plot_duration_distribution(durations)

In [ ]:
import torch
a = torch.tensor([[1,2],
                  [3,4]])
b = torch.tensor([[5,6],
                 [7,8]])
torch.maximum(a, a.max() - 1)

In [ ]:
from torchaudio import transforms
import torchaudio

test_flac = '/Users/lkieu/PycharmProjects/PhonemeAwareFoundational/test_audio/canterburytales_09_chaucer_64kb.flac'
waveform, sample_rate = torchaudio.load(test_flac, normalize=True)
transform = transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=1024, hop_length=256, n_mels=80)
melspec = transform(waveform)
# Define max size (in time frames)
max_frames = 500  # Example

def crop_or_pad(spec, max_frames):
    channels, n_mels, time_frames = spec.shape
    if time_frames > max_frames:
        return spec[:, :, :max_frames]
    elif time_frames < max_frames:
        pad_amount = max_frames - time_frames
        pad = torch.zeros((channels, n_mels, pad_amount), device=spec.device)
        return torch.cat((spec, pad), dim=2)
    else:
        return spec

melspec= crop_or_pad(melspec, max_frames)
print(melspec.shape)  # Should be (channels, n_mels, max_frames)


In [ ]:
from src.data.util import log_compress
import librosa

log_spec_lib = librosa.power_to_db(melspec[0])
log_spec_torch = log_compress(melspec[0])

In [ ]:
import matplotlib.pyplot as plt

def plot_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(specgram, origin="lower", aspect="auto", interpolation="nearest")

In [ ]:
plot_spectrogram(log_spec_lib)

In [ ]:
plot_spectrogram(log_spec_torch)

In [ ]:
from src.data.util import whisper_norm, z_score_norm, min_max_norm
whisper_norm_log_spec = whisper_norm(log_spec_torch)
plot_spectrogram(whisper_norm_log_spec)

In [ ]:
z_norm = z_score_norm(log_spec_torch)
plot_spectrogram(z_norm)

In [ ]:
min_max_norm_log_spec = min_max_norm(log_spec_torch)
plot_spectrogram(min_max_norm_log_spec)